# PA005: High Value Customer Identification (Insiders)

# 0.0 Imports

In [1]:
import re
import os
import s3fs
import pickle
import inflection
import numpy   as np
import pandas  as pd
import seaborn as sns
import umap.umap_ as umap

from matplotlib import pyplot as plt

from sklearn import cluster as c
from sklearn import mixture as mx
from sklearn import metrics as m
from sklearn import decomposition as dd
from sklearn.manifold import TSNE
from sklearn import ensemble as en
from sklearn import preprocessing as pp

from plotly import express as px

In [3]:
AWS_ACCESS_KEY_ID = os.environ.get('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.environ.get('AWS_SECRET_ACCESS_KEY')

## 0.2 Load dataset

In [4]:
#load data
df = pd.read_csv('../data/Ecommerce.csv', encoding='unicode_escape')

#drop extra column
df = df.drop(columns=['Unnamed: 8'], axis=1)

# 1.0 Descrição dos dados

In [5]:
df1 = df.copy()

## 1.1 Rename Columns

In [6]:
cols_old = ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
            'UnitPrice', 'CustomerID', 'Country']

snakecase = lambda x: inflection.underscore( x )

cols_new = list(map(snakecase, cols_old))

#rename
df1.columns = cols_new

## 1.2 Data Dimensions

In [7]:
print('Number of Rows: {}'.format(df1.shape[0]))
print('Number of Columns: {}'.format(df1.shape[1]))

Number of Rows: 541909
Number of Columns: 8


## 1.3 Data Types

In [8]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
customer_id     float64
country          object
dtype: object

## 1.4 Check NA

In [9]:
df1.isnull().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

## 1.5 Replace NA

In [10]:
#Separar os Dados com NA
df_missing = df1.loc[df1['customer_id'].isna(), :]
df_not_missing = df1.loc[~df1['customer_id'].isna(), :]

In [11]:
df_not_missing.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.0,United Kingdom


In [12]:
#Teste para identificar se os invoice_no correspondem em ambos os df's. Nesse caso, não há correspondência. Será necessária outra estratégia.
missing_invoice = df_missing['invoice_no'].drop_duplicates().tolist()
df_not_missing.loc[df_not_missing['invoice_no'].isin(missing_invoice), :]

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country


In [13]:
#Verificar o valor máximo do 'customer_id'
df_not_missing['customer_id'].max()

18287.0

In [14]:
#Para não perder os dados, atribuir valores de 'customer_id' para os 'invoice_no' do df com NA's. Usar esses dados para treinar o algoritmo e retira-los ao final da modelagem.
#Create reference
df_backup = pd.DataFrame(df_missing['invoice_no'].drop_duplicates())
df_backup['customer_id'] = np.arange(19000, 19000+len(df_backup), 1)

# merge original with reference dataframe
df1 = pd.merge(df1, df_backup, on='invoice_no', how='left')

#Coalesce
## Come foram geradas duas colunas e ambas possuem NA's, será necessário usar artifício para combinar as colunas 
df1['customer_id'] = df1['customer_id_x'].combine_first(df1['customer_id_y'])

#drop extra columns
df1 = df1.drop(columns=['customer_id_x', 'customer_id_y'], axis=1)

df1.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,United Kingdom,17850.0
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,United Kingdom,17850.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,United Kingdom,17850.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,United Kingdom,17850.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,United Kingdom,17850.0


In [15]:
df1.isnull().sum()

invoice_no         0
stock_code         0
description     1454
quantity           0
invoice_date       0
unit_price         0
country            0
customer_id        0
dtype: int64

## 1.6 Change Dtypes

In [16]:
df1.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,United Kingdom,17850.0
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,United Kingdom,17850.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,United Kingdom,17850.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,United Kingdom,17850.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,United Kingdom,17850.0


In [17]:
#invoice_no e stock_code serão analisados posteriormente

#invoice_date
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'], format = '%d-%b-%y')

#customer_id
df1['customer_id'] = df1['customer_id'].astype('int64')

In [18]:
df1.dtypes

invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
country                 object
customer_id              int64
dtype: object

## 1.7 Descriptive Statistics

In [19]:
num_attributes = df1.select_dtypes(include=['int64', 'float64'])
cat_attributes = df1.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]'])

### 1.7.1 Numerical Attributes

In [20]:
# Central Tendency - mean, median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

# dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(min)).T
d3 = pd.DataFrame(num_attributes.apply(max)).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

#Concatenar
m1 = pd.concat( [d2, d3, d4, ct1, ct2, d1, d5, d6] ).T.reset_index()
m1.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
m1

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,quantity,-80995.00,80995.0,161990.00,9.552250,3.00,218.080957,-0.264076,119769.160031
1,unit_price,-11062.06,38970.0,50032.06,4.611114,2.08,96.759764,186.506972,59005.719097
2,customer_id,12346.00,22709.0,10363.00,16688.840453,16249.00,2911.408666,0.487449,-0.804287


### 1.7.2 Categorical Attributes

In [21]:
#invoice_no

#cat_attributes['invoice_no'].astype(int) - #Algumas strings entre os números
#len(cat_attributes.loc[cat_attributes['invoice_no'].apply(lambda x: bool(re.search( '[^0-9]+', x ))), 'invoice_no'].drop_duplicates())

df_invoices = df1.loc[df1['invoice_no'].apply(lambda x: bool(re.search( '[^0-9]+', x ))), :]

print('Total number of invoices: {}'.format(len(df_invoices)))
print('Total number of negative quantity: {}'.format(len(df_invoices[df_invoices['quantity'] < 0])))

Total number of invoices: 9291
Total number of negative quantity: 9288


In [22]:
#stock_code
#aux = len(cat_attributes.loc[cat_attributes['stock_code'].apply(lambda x: bool(re.search( '[^0-9]+', x))), 'stock_code'].drop_duplicates())

#cat_attributes.loc[cat_attributes['stock_code'].apply(lambda x: bool(re.search( '^[a-zA-Z]+$', x))), 'stock_code'].unique()
#df1.loc[cat_attributes['stock_code'].apply(lambda x: bool(re.search( '^[a-zA-Z]$', x ))), :].head()

df1.loc[df1['stock_code'].apply(lambda x: bool(re.search('^[a-zA-Z]+$', x))), 'stock_code'].unique()

#Ação:
## 1. Remove stock_code in ['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY','DCGSSGIRL', 'PADS', 'B', 'CRUK']

array(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY',
       'DCGSSGIRL', 'PADS', 'B', 'CRUK'], dtype=object)

### Description

In [23]:
df1.head()

#Ação: Delete description

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2016-11-29,2.55,United Kingdom,17850
1,536365,71053,WHITE METAL LANTERN,6,2016-11-29,3.39,United Kingdom,17850
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2016-11-29,2.75,United Kingdom,17850
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2016-11-29,3.39,United Kingdom,17850
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2016-11-29,3.39,United Kingdom,17850


### Country

In [24]:
df1['country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [25]:
df1['country'].value_counts(normalize=True).head()

United Kingdom    0.914320
Germany           0.017521
France            0.015790
EIRE              0.015124
Spain             0.004674
Name: country, dtype: float64

In [26]:
#Drop_duplicates é usado quando quero saber o número de usuários únicos. Sem o drop a granularidade é por compra. Ex: 1 usuário pode ter feito 10 compras
df1[['customer_id', 'country']].drop_duplicates().groupby('country').count().reset_index().sort_values('customer_id', ascending=False).head()

,country,customer_id
36,United Kingdom,7587
14,Germany,95
13,France,90
10,EIRE,44
31,Spain,31


# 2.0 Filtragem de Variáveis

- Nesse caso a filtragem se fez necessária antes do passo 2 pq no passo 2 são feitos cálculos pra criar as variáveis, logo, n pode haver sujeira. Isso justifica adiantar o passo 3

In [27]:
df2 = df1.copy()

In [28]:
# Numerical Attributes

## Separar entre dados com devoluções e sem devoluções
df2 = df2.loc[df2['unit_price'] >= 0.04, :]


# -- Categorical Attributes --

#isin pq fica mais fácil já que é uma lista | ~ (usado com o isin) pq vou pegar oq não estiver contido na condição
df2 = df2[~df2['stock_code'].isin(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK'])] 

#description
df2 = df2.drop(columns='description', axis=1)

#map 
df2 = df2[~df2['country'].isin(['European Community', 'Unspecified'])]

#bad users
df2 = df2[~df2['customer_id'].isin([16446])]

#quantity
df2_charge_back = df2.loc[df2['quantity'] < 0, :]
df2_purchases = df2.loc[df2['quantity'] >= 0, :]


# 3.0 Feature Engeneering

In [29]:
df3 = df2.copy()

## 3.1 Feature Creation

In [30]:
# data reference
df_ref = df3.drop(['invoice_no', 'stock_code', 'quantity', 'invoice_date',
                   'unit_price', 'country'], axis=1).drop_duplicates(ignore_index=True)

- Ficar atento pq Meiga não usou o purchases pra fazer o df_ref

### 3.1.1 Gross Revenue

In [31]:
#Gross Revenue (Faturamento) quantity * price
df2_purchases.loc[:, 'gross_revenue'] = df2_purchases.loc[:, 'quantity'] * df2_purchases.loc[:, 'unit_price']

#Monetary
df_monetary = df2_purchases.loc[: ,['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge(df_ref, df_monetary, on='customer_id', how='left')

/tmp/ipykernel_42780/1095068563.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_purchases.loc[:, 'gross_revenue'] = df2_purchases.loc[:, 'quantity'] * df2_purchases.loc[:, 'unit_price']


### 3.1.2 Recency

In [32]:
#Recency - Last day purchase - Como o dataset é antigo, usar como date o último invoice_date
df_recency = df2_purchases[['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index() # Agrupa por usuários e pega a data máxima da última compra de cada
df_recency['recency_days'] = (df2_purchases['invoice_date'].max() - df_recency['invoice_date']).dt.days
df_recency = df_recency[['customer_id', 'recency_days']].copy()
df_ref = pd.merge(df_ref, df_recency, on='customer_id', how='left')

### 3.1.3 Quantity purchases

In [33]:
#Numero de compras no ano
df_frequency = df2_purchases[['customer_id', 'invoice_no']].drop_duplicates().groupby('customer_id').count().reset_index().rename(columns={'invoice_no': 'qtd_invoices'})
df_ref = pd.merge(df_ref, df_frequency, on='customer_id', how='left')

### 3.1.4 Quantity of items purchased

In [34]:
#Número de produtos comprados
## Com o drop_duplicates os valores de quantity que fossem iguais estavam sendo descartados, gerando incongruência nos dados.
#df_frequency = df2_purchases[['customer_id', 'quantity']].drop_duplicates().groupby('customer_id').count().reset_index()

df_frequency = df2_purchases[['customer_id','quantity']].groupby('customer_id').sum().reset_index().rename(columns={'quantity': 'qtd_items'})
df_ref = pd.merge(df_ref, df_frequency, on='customer_id', how='left')

### 3.1.4 Quantity type of items purchased

In [35]:
df_frequency = df2_purchases[['customer_id','stock_code']].groupby('customer_id').count().reset_index().rename(columns={'stock_code': 'qtd_products'})
df_ref = pd.merge(df_ref, df_frequency, on='customer_id', how='left')

### 3.1.5 Average Ticket Value

In [36]:
#AVG Ticket
df_avg_ticket = df2_purchases[['customer_id', 'gross_revenue']].groupby('customer_id').mean().reset_index().rename(columns={'gross_revenue': 'avg_ticket'})
df_ref = pd.merge(df_ref, df_avg_ticket, on='customer_id', how='left') #Sempre calcular os NA's depois do left join
df_ref.isnull().sum()

customer_id       0
gross_revenue    91
recency_days     91
qtd_invoices     91
qtd_items        91
qtd_products     91
avg_ticket       91
dtype: int64

### 3.1.6 Average Recency Days

In [37]:
#Descobrir a diferença de dias entre as datas das compras
df_aux = df2[['customer_id', 'invoice_date']].drop_duplicates().sort_values(['customer_id', 'invoice_date'])
df_aux['next_customer_id'] = df_aux['customer_id'].shift() # 'shift' desloca a coluna selecionada para 1 linha abaixo
df_aux['previous_date'] = df_aux['invoice_date'].shift()

#Com as colunas deslocadas, é possível fazer uma subtração na mesma linha
df_aux['avg_recency_days'] = df_aux.apply(lambda x: (x['invoice_date'] - x['previous_date']).days if x['customer_id'] == x['next_customer_id'] else np.nan, axis =1) #'axis=1' pra + 2 colun

#Dropa as colunas que n vão ser usadas e as linhas que contiverem NA's
df_aux = df_aux.drop(['invoice_date', 'next_customer_id', 'previous_date'], axis=1).dropna()

#Average recency
df_avg_recency_days = df_aux.groupby('customer_id').mean().reset_index()

#merge

df_ref = pd.merge(df_ref, df_avg_recency_days, on='customer_id', how='left')
df_ref.isna().sum()

customer_id            0
gross_revenue         91
recency_days          91
qtd_invoices          91
qtd_items             91
qtd_products          91
avg_ticket            91
avg_recency_days    2816
dtype: int64

### 3.1.7 Frequency Purchase

In [38]:
#Pegar a data máxima, mínima, subtrair e dividir pela quantidade de compras
df2_max = df2[['customer_id', 'invoice_date']].drop_duplicates().groupby('customer_id').max().reset_index()
df2_min = df2[['customer_id', 'invoice_date']].drop_duplicates().groupby('customer_id').min().reset_index()
df2_purchase = df2[['customer_id', 'invoice_date']].drop_duplicates().groupby('customer_id').count().reset_index()


In [39]:
#Função agregate (nome da coluna, coluna que será utilizada, operação nessa coluna)
df_aux = (df2_purchases[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates().groupby('customer_id')
                                                                     .agg(max_ = ('invoice_date', 'max'), #Apenas para melhorar a visualização, pois o cálculo é feito em "days_"
                                                                          min_ = ('invoice_date', 'min'),
                                                                          days_= ('invoice_date', lambda x: ((x.max() - x.min()).days) + 1),
                                                                          buy_ = ('invoice_no', 'count'))).reset_index()

#Calculate Frequency
df_aux['frequency'] = df_aux[['buy_', 'days_']].apply( lambda x: x['buy_'] / x['days_'] if x['days_'] != 0 else 0, axis=1)

#merge
df_ref = pd.merge(df_ref, df_aux[['customer_id', 'frequency']], on='customer_id', how='left')
df_ref.isna().sum()

customer_id            0
gross_revenue         91
recency_days          91
qtd_invoices          91
qtd_items             91
qtd_products          91
avg_ticket            91
avg_recency_days    2816
frequency             91
dtype: int64

### 3.1.8 Devoluções

In [40]:
df_returns = df2_charge_back[['customer_id', 'quantity']].groupby('customer_id').sum().reset_index().rename(columns={'quantity': 'qtd_returns'})
df_returns['qtd_returns'] = df_returns['qtd_returns'] * -1

#merge
df_ref = pd.merge(df_ref, df_returns, on='customer_id', how='left')

#Substituir NA's returns por 0
df_ref.loc[df_ref['qtd_returns'].isna(), 'qtd_returns'] = 0
#df_ref['qtd_returns'].fillna(0, inplace = True)

df_ref.isna().sum()

customer_id            0
gross_revenue         91
recency_days          91
qtd_invoices          91
qtd_items             91
qtd_products          91
avg_ticket            91
avg_recency_days    2816
frequency             91
qtd_returns            0
dtype: int64

### 3.1.9 Basket Size

In [41]:
#Criação de duas colunas: número de compras e quantidade total de produtos por usuário
df_aux = (df2_purchases.loc[:, ['customer_id', 'invoice_no', 'quantity']].groupby('customer_id')
                                                                 .agg(n_purchase=('invoice_no', 'nunique'),
                                                                      n_products=('quantity', 'sum')).reset_index())

#Cálculo da média de itens por compras
df_aux['avg_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

#merge
df_ref = pd.merge(df_ref, df_aux[['customer_id', 'avg_basket_size']], on='customer_id', how='left')

### 3.1.10 Unique Basket Size - Quantidade de produtos distintos por compra

In [42]:
#Criação de duas colunas: número de compras e produtos distintos por usuário
df_aux = (df2_purchases.loc[:, ['customer_id', 'invoice_no', 'stock_code']].groupby('customer_id')
                                                                 .agg(n_purchases=('invoice_no', 'nunique'),
                                                                      n_stock_code=('stock_code', 'nunique')).reset_index())

#Cálculo da média de itens distintos por compras
df_aux['avg_unique_basket_size'] = df_aux['n_stock_code'] / df_aux['n_purchases']


#merge
df_ref = pd.merge(df_ref, df_aux[['customer_id', 'avg_unique_basket_size']], on='customer_id', how='left')

In [43]:
df_ref.isna().sum()

customer_id                  0
gross_revenue               91
recency_days                91
qtd_invoices                91
qtd_items                   91
qtd_products                91
avg_ticket                  91
avg_recency_days          2816
frequency                   91
qtd_returns                  0
avg_basket_size             91
avg_unique_basket_size      91
dtype: int64

# 4.0 EDA (Exploratory Data Analysis)

In [44]:
df4 = df_ref.dropna()
df4.isnull().sum()

customer_id               0
gross_revenue             0
recency_days              0
qtd_invoices              0
qtd_items                 0
qtd_products              0
avg_ticket                0
avg_recency_days          0
frequency                 0
qtd_returns               0
avg_basket_size           0
avg_unique_basket_size    0
dtype: int64

## 4.3 Estudo do Espaço

In [45]:
#Selected dataset
cols_selected = ['customer_id', 'gross_revenue', 'recency_days', 'qtd_products', 'frequency', 'qtd_returns']
df43 = df4[cols_selected].copy()

In [46]:
mm = pp.MinMaxScaler()
fs = s3fs.S3FileSystem(anon=False, key=AWS_ACCESS_KEY_ID, secret=AWS_SECRET_ACCESS_KEY) #anon - se é anônimo ou n

#
df43['gross_revenue']          = mm.fit_transform(df43[['gross_revenue']])
pickle.dump(mm, open('../src/features/gross_revenue_scaler.pkl', 'wb'))
pickle.dump(mm, fs.open('s3://insider-dataset/gross_revenue_scaler.pkl', 'wb'))

df43['recency_days']           = mm.fit_transform(df43[['recency_days']])
pickle.dump(mm, open('../src/features/recency_days_scaler.pkl', 'wb'))
pickle.dump(mm, fs.open('s3://insider-dataset/recency_days_scaler.pkl', 'wb'))

df43['qtd_products']           = mm.fit_transform(df43[['qtd_products']])
pickle.dump(mm, open('../src/features/qtd_products_scaler.pkl', 'wb'))
pickle.dump(mm, fs.open('s3://insider-dataset/qtd_products_scaler.pkl', 'wb'))

df43['frequency']              = mm.fit_transform(df43[['frequency']])
pickle.dump(mm, open('../src/features/frequency_scaler.pkl', 'wb'))
pickle.dump(mm, fs.open('s3://insider-dataset/frequency_scaler.pkl', 'wb'))

df43['qtd_returns']            = mm.fit_transform(df43[['qtd_returns']])
pickle.dump(mm, open('../src/features/qtd_returns_scaler.pkl', 'wb'))
pickle.dump(mm, fs.open('s3://insider-dataset/qtd_returns_scaler.pkl', 'wb'))


### 4.3.4 Tree-Based Embedding

In [47]:
#Como o problema é não supervisionado, usei a variável gross revenue para ser a variável resposta, tendo em vista que pra esse problema de negócio esta variável é a mais importante
##Como o problema é de árvore, n é necessário usar os dados com reescala

#training dataset
X = df43.drop(columns=['customer_id', 'gross_revenue'], axis=1)
y = df43['gross_revenue']

#model definition
rf_model = en.RandomForestRegressor( n_estimators=100, random_state=42)

#model training
rf_model.fit(X, y)

pickle.dump(rf_model, open('../src/models/rf_model.pkl', 'wb'))
pickle.dump(rf_model, fs.open('s3://insider-dataset/rf_model.pkl', 'wb'))

#Aplicar os dados de treino sobre a árvore treinada com o objetivo de obter as folhas 
df_leaf = pd.DataFrame(rf_model.apply(X))



In [48]:
#Reduzir dimensionalidade de 100 para 2 e ser possível visualizar 
reducer = umap.UMAP(random_state=42)
embedding = reducer.fit_transform(df_leaf)

pickle.dump(reducer, open('../src/features/umap_reducer.pkl', 'wb'))
pickle.dump(reducer, fs.open('s3://insider-dataset/umap_reducer.pkl', 'wb'))

#embedding
df_tree = pd.DataFrame()
df_tree['embedding_x'] = embedding[:, 0]
df_tree['embedding_y'] = embedding[:, 1]

# 7.0 Hyperparameter Fine-Tunning

In [47]:
X = df_tree.copy()

In [48]:
X.head()

,embedding_x,embedding_y
0,10.918785,14.602229
1,19.531637,8.913908
2,11.945412,14.611155
3,0.142954,2.149642
4,-6.805303,9.022922


# 8.0 Model Training

## 8.1 K-Means

In [62]:
#model definition
k=8

#model definition
gmm_model = mx.GaussianMixture(n_components=k, n_init=300, random_state=32) 

#model training
gmm_model.fit(X)

#model predict
labels = gmm_model.predict(X)


### 8.2 Cluster Validation

In [63]:
#WSS
#print('WSS value: {}'.format(kmeans.inertia_))

#SS
print(m.silhouette_score(X, labels, metric='euclidean'))

0.51884824


# 9.0 Cluster Analysis

In [64]:
df92 = df4[cols_selected].copy()
df92['cluster'] = labels

#change types
df92['recency_days'] = df92['recency_days'].astype('int64')
df92['qtd_products'] = df92['qtd_products'].astype('int64')
df92['qtd_returns'] = df92['qtd_returns'].astype('int64')


In [65]:
df92.head()

,customer_id,gross_revenue,recency_days,qtd_products,frequency,qtd_returns,cluster
0,17850,5391.21,372,297,17.000000,40,6
1,13047,3232.59,56,171,0.028302,35,3
2,12583,6705.38,2,232,0.040323,50,6
3,13748,948.25,95,28,0.017921,0,3
4,15100,876.00,333,3,0.073171,22,7


In [66]:
# Number of customer
df_cluster = df92[['customer_id', 'cluster']].groupby('cluster').count().reset_index()
df_cluster['perc_customer'] = 100 * (df_cluster['customer_id'] / df_cluster['customer_id'].sum())

#AVG Gross revenue
df_avg_gross_revenue = df92[['gross_revenue', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_gross_revenue, how='inner', on='cluster')

#AVG Recency
df_avg_recency = df92[['recency_days', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_recency, how='inner', on='cluster')

#AVG Frequency
df_avg_qtd_products = df92[['qtd_products', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_qtd_products, how='inner', on='cluster')

#AVG Frequency
df_avg_frequency = df92[['frequency', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_frequency, how='inner', on='cluster')

#Returns
df_avg_qtd_returns = df92[['qtd_returns', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_qtd_returns, how='inner', on='cluster')

df_cluster.sort_values('gross_revenue', ascending=False)

,cluster,customer_id,perc_customer,gross_revenue,recency_days,qtd_products,frequency,qtd_returns
6,6,373,12.567385,10503.060483,19.268097,475.581769,0.110615,187.120643
5,5,95,3.200809,2291.266000,29.957895,222.136842,0.031576,0.663158
4,4,349,11.758760,2216.344413,69.501433,52.896848,0.072810,11.028653
3,3,592,19.946092,2083.700878,51.005068,109.479730,0.077666,19.861486
1,1,488,16.442049,1861.938012,58.532787,95.969262,0.054101,15.159836
0,0,479,16.138814,1284.840418,62.194154,57.154489,0.048812,14.225470
2,2,200,6.738544,647.933500,47.280000,11.510000,0.025991,0.845000
7,7,392,13.207547,505.728827,149.426020,15.522959,0.426596,9.443878


### Cluster 01: (Candidato à Insider)
    - Número de customers: 6 (0.2% dos customers)
    - Recência em média: 7 dias
    - Compras em média: 89 compras
    - Receita em média: $182.182,00 dólares
        
### Cluster 02: 
    - Número de customers: 31 (0.8% dos customers)
    - Recência em média: 14 dias
    - Compras em média:53 compras
    - Receita em média: $40.543,52 dólares
        
### Cluster 03: 
    - Número de customers: 4.335 (99% dos customers)
    - Recência em média: 92 dias
    - Compras em média:5 compras
    - Receita em média: $1.372,57 dólares

# 11.0 Deploy To Production

In [53]:
df92.dtypes

customer_id        int64
gross_revenue    float64
recency_days       int64
qtd_products       int64
frequency        float64
qtd_returns        int64
cluster            int64
dtype: object

## 11.1 Insert into SQLITE

In [54]:
import sqlite3
from sqlalchemy import create_engine

In [55]:
#create table
query_create_table_insiders = """
    CREATE TABLE insiders (
        customer_id     INTEGER,
        gross_revenue   REAL,
        recency_days    INTEGER,
        qtd_products    INTEGER,
        frequency       REAL,
        qtd_returns     INTEGER,
        cluster         INTEGER  
    )

"""

conn = sqlite3.connect('insiders_db.sqlite')
conn.execute(query_create_table_insiders)
conn.commit()
conn.close()

#insert data
conn = create_engine('sqlite:///insiders_db.sqlite') #Conexão com o banco de dados
df92.to_sql('insiders', con=conn, if_exists='append', index=False)

OperationalError: table insiders already exists

In [ ]:
#consulting database
query = """
    SELECT * FROM insiders
"""

df = pd.read_sql_query(query, conn)

In [ ]:
df.head()

In [ ]:
df.shape